# Machine Learning to identify whether a vehicle is going to a correct path
# Kindly take reference to the image given in the same folder 
##First to generate desired GPS trails I have used https://www.findlatitudeandlongitude.com/click-lat-lng-list/#.WaalPtMjEUs
##As you can see in Raw_Data11.xlsx file the two successive columns are considered one GPS trail as you can see below
##Thus the following Data have 10 GPS trail among which first 7 are for one path that the cab driver use to take and another 3 are for deviated path that it will take
##The reason behind taking both the path during learning phase is that machine must have to know the desired path's that it can take during his journey


# Reading the Data from the Excel file(Raw_Data11.xlsx) and Naming the columns as ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t"]


In [75]:
import pandas as pd
Raw_Data = pd.ExcelFile("Raw_Data11.xlsx")
print(Raw_Data.sheet_names)
Data_Fram = Raw_Data.parse("Sheet 1")
print(Data_Fram)

['Sheet 1', 'Sheet 2']
      Table 1  Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  \
0         NaN         NaN         NaN         NaN         NaN         NaN   
1   18.592725   73.819842   18.592765   73.819585   18.592752   73.819571   
2   18.592074   73.818984   18.592074   73.818727   18.592142   73.818841   
3   18.591586   73.817954   18.591260   73.817353   18.591735   73.818111   
4   18.591098   73.816838   18.590243   73.816752   18.591165   73.817339   
5   18.589755   73.816710   18.588860   73.816667   18.590108   73.817081   
6   18.588128   73.816581   18.587518   73.816366   18.588033   73.816652   
7   18.586176   73.816237   18.586013   73.816366   18.585959   73.816223   
8   18.584020   73.815894   18.584549   73.815894   18.584088   73.815837   
9   18.581864   73.815551   18.583084   73.815722   18.582176   73.815622   
10  18.579993   73.815894   18.581742   73.815594   18.580589   73.815751   
11  18.577877   73.815937   18.580237   73.815937   1

In [76]:
Data = Data_Fram[1:]
Data.columns = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t"]
print(Data)

            a          b          c          d          e          f  \
1   18.592725  73.819842  18.592765  73.819585  18.592752  73.819571   
2   18.592074  73.818984  18.592074  73.818727  18.592142  73.818841   
3   18.591586  73.817954  18.591260  73.817353  18.591735  73.818111   
4   18.591098  73.816838  18.590243  73.816752  18.591165  73.817339   
5   18.589755  73.816710  18.588860  73.816667  18.590108  73.817081   
6   18.588128  73.816581  18.587518  73.816366  18.588033  73.816652   
7   18.586176  73.816237  18.586013  73.816366  18.585959  73.816223   
8   18.584020  73.815894  18.584549  73.815894  18.584088  73.815837   
9   18.581864  73.815551  18.583084  73.815722  18.582176  73.815622   
10  18.579993  73.815894  18.581742  73.815594  18.580589  73.815751   
11  18.577877  73.815937  18.580237  73.815937  18.579043  73.816009   
12  18.575843  73.815637  18.578691  73.816109  18.577538  73.815794   
13  18.575437  73.814178  18.577348  73.815680  18.576033  73.81

# Now the data-frame is converted to a list within list's and each list identify some coordinate on map
#As the successive lists are generated horizontally among first row in our data frame ie there are 20 columns and each two column of first row gives us an coordinate on map that means we have 10 coordinates among which we have first 7 are from one path and other 3 are from deviated path, so that's how the below list is generated 

In [77]:
from sklearn import svm
import numpy as np
x = Data.values.tolist()
initial_data = []
final_data = []
for row in x:
    initial_data.append([row[i:i+2] for i  in range(0, len(row), 2)])
for y in initial_data:
    for z in y:
        final_data.append(z)
print(final_data)

[[18.592725, 73.819842], [18.592765, 73.819585], [18.592752, 73.819571], [18.592806, 73.819499], [18.592887, 73.819542], [18.592887, 73.819585], [18.592806, 73.819714], [18.592725, 73.819263], [18.592786, 73.819349], [18.56726, 73.81568], [18.592074, 73.818984], [18.592074, 73.818727], [18.592142, 73.818841], [18.591952, 73.818512], [18.592236, 73.818855], [18.592236, 73.818812], [18.592114, 73.818898], [18.592765, 73.819778], [18.592033, 73.819714], [18.592887, 73.81937], [18.591586, 73.817954], [18.59126, 73.817353], [18.591735, 73.818111], [18.591301, 73.817482], [18.591586, 73.818083], [18.591586, 73.81804], [18.591626, 73.818083], [18.592236, 73.819864], [18.590853, 73.819563], [18.591748, 73.819799], [18.591098, 73.816838], [18.590243, 73.816752], [18.591165, 73.817339], [18.590284, 73.816838], [18.590935, 73.816967], [18.591057, 73.817225], [18.591138, 73.817225], [18.591545, 73.819714], [18.589776, 73.819671], [18.590528, 73.819542], [18.589755, 73.81671], [18.58886, 73.816667]

# Clustering of Data is to be done for learning process
#As I mentioned above the first 7 coordinate belong to one path which is clustered to "0" cluster and next 3 belongs to cluster on "1"
#So the desired output is generated using nested for loop

In [86]:
y = []
for val in range(int(len(final_data)/10)):
    for x in range(10):
        if(x>6):
            y.append(1)
        else:
            y.append(0)
print(y)

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]


# K nearest neighbor Algorithm is used for learning and prediction

In [79]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(final_data,y) 
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

# As the learning is completed now we can run the test to get the desired result


In [80]:
import pandas as pd
Raw_Data = pd.ExcelFile("Raw_Data11.xlsx")
print(Raw_Data.sheet_names)
Data_Fram = Raw_Data.parse("Sheet 2")
print(Data_Fram)

['Sheet 1', 'Sheet 2']
      Table 1  Unnamed: 1
0   18.592752   73.819313
1   18.591491   73.819699
2   18.589986   73.819613
3   18.588806   73.820043
4   18.587626   73.820601
5   18.586447   73.821158
6   18.585552   73.821974
7   18.584494   73.822961
8   18.583437   73.824120
9   18.581850   73.822789
10  18.580020   73.820171
11  18.577986   73.819699
12  18.576318   73.819656
13  18.575789   73.818026
14  18.575382   73.816094
15  18.575504   73.813648
16  18.575952   73.811159
17  18.575423   73.809056
18  18.574203   73.807726
19  18.572494   73.807082
20  18.570826   73.808413
21  18.569443   73.810430
22  18.567612   73.811932
23  18.565375   73.812575
24  18.563137   73.813219
25  18.560900   73.814163
26  18.558744   73.815966
27  18.556587   73.817768
28  18.554431   73.819571
29  18.552275   73.821373


In [81]:
test_Data = Data_Fram[1:]
test_Data.columns = ["a","b"]
print(test_Data)


            a          b
1   18.591491  73.819699
2   18.589986  73.819613
3   18.588806  73.820043
4   18.587626  73.820601
5   18.586447  73.821158
6   18.585552  73.821974
7   18.584494  73.822961
8   18.583437  73.824120
9   18.581850  73.822789
10  18.580020  73.820171
11  18.577986  73.819699
12  18.576318  73.819656
13  18.575789  73.818026
14  18.575382  73.816094
15  18.575504  73.813648
16  18.575952  73.811159
17  18.575423  73.809056
18  18.574203  73.807726
19  18.572494  73.807082
20  18.570826  73.808413
21  18.569443  73.810430
22  18.567612  73.811932
23  18.565375  73.812575
24  18.563137  73.813219
25  18.560900  73.814163
26  18.558744  73.815966
27  18.556587  73.817768
28  18.554431  73.819571
29  18.552275  73.821373


In [82]:
from sklearn import svm
import numpy as np
x = test_Data.values.tolist()
initial_data = []
final_test_data = []
for row in x:
    initial_data.append([row[i:i+2] for i  in range(0, len(row), 2)])
for y in initial_data:
    for z in y:
        final_test_data.append(z)
print(final_test_data)

[[18.591491, 73.819699], [18.589986, 73.819613], [18.588806, 73.820043], [18.587626, 73.820601], [18.586447, 73.821158], [18.585552, 73.821974], [18.584494, 73.822961], [18.583437, 73.82412], [18.58185, 73.822789], [18.58002, 73.820171], [18.577986, 73.819699], [18.576318, 73.819656], [18.575789, 73.818026], [18.575382, 73.816094], [18.575504, 73.813648], [18.575952, 73.811159], [18.575423, 73.809056], [18.574203, 73.807726], [18.572494, 73.807082], [18.570826, 73.808413], [18.569443, 73.81043], [18.567612, 73.811932], [18.565375, 73.812575], [18.563137, 73.813219], [18.5609, 73.814163], [18.558744, 73.815966], [18.556587, 73.817768], [18.554431, 73.819571], [18.552275, 73.821373]]


# Now the output is an array of 0 and 1 means in the above list or coordinates first 13 coordinates belong to 1 cluster ie it is traveling from the deviated path and after some point, it will come back to the first path

In [85]:
knn.predict(final_test_data)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])